In [47]:
import pandas as pd
import numpy as np
from itertools import product
from interpret.glassbox import ExplainableBoostingClassifier as ebc
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score as roc
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from interpret import show

## Housekeeping

In [22]:
data_dir = "."
df = pd.read_csv("{}/mimic4_flat_large.csv".format(data_dir))
df_cols = df.columns.tolist()
looking_for = ['heartrate', 'temp', 'systolic', 'diastolic',
              'spo2', 'glucose', 'albumin', 'bicarbonate', 'bilirubin',
              'creatinine', 'chloride', 'hematocrit', 'hemoglobin', 'lactate',
              'magnesium', 'phosphate', 'platelet', 'potassium', 'ptt', 'sodium', 'bun', 'wbc']
possible_cols = []
for lab_type in df_cols:
    for val in looking_for:
        if val in lab_type.lower():
            possible_cols.append(lab_type)
lab_cols = [
    'ART BP Diastolic',
    'ART BP Systolic',
    'Albumin',
    'BUN',
    'Calcium Chloride',
    'Chloride (serum)',
    'Creatinine (serum)',
    'Glucose (serum)',
    'Hematocrit (serum)',
    'Hemoglobin',
    'Magnesium',
    'PTT',
    'Platelet Count',
    'Potassium (serum)',
    'Sodium (serum)',
    'Temperature Fahrenheit',
    'Total Bilirubin',
    'WBC',]
df.drop('Communication', axis=1, inplace=True)
df_cols = df.columns.tolist()
demo_cols = ['admission_type', 'insurance', 'marital_status', 'ethnicity', 'gender', 'age']
treatment_cols = df_cols[df_cols.index('mortality')+1:df_cols.index('Insulin - Novolog')+1]
treatment_cols = np.array(treatment_cols)[np.sum(df[treatment_cols], axis=0) > 100]

X_demo = df[demo_cols]
X_treatments = df[treatment_cols]
X_labs = df[lab_cols]
fill_values = {}
for i in range(X_demo.shape[1]):
    if X_demo.values[0, i].__class__ == str:
        fill_values[i] = 'Missing'
    elif X_demo.values[0, i].__class__ == bool:
        fill_values[i] = False
    elif X_demo.values[0, i].__class__ == float or X_demo.values[0, i].__class__ == int:
        fill_values[i] = -1
for j in range(X_labs.shape[1]):
    if X_labs.values[0, j].__class__ == str:
        fill_values[i+j] = 'Missing'
    elif X_labs.values[0, j].__class__ == bool:
        fill_values[i+j] = False
    elif X_labs.values[0, j].__class__ == float or X_labs.values[0, j].__class__ == int:
        fill_values[i+j] = -1
C = pd.concat([X_demo, X_labs], axis=1).fillna(value=fill_values)

for j in range(X_treatments.shape[1]):
    if X_treatments.values[0, j].__class__ == str:
        fill_values[j] = 'Missing'
    elif X_treatments.values[0, j].__class__ == bool:
        fill_values[j] = False
    elif X_treatments.values[0, j].__class__ == float or X_treatments.values[0, j].__class__ == int:
        fill_values[j] = -1
X = X_treatments.fillna(value=fill_values)

def fillna_unknown_dtype_col(X):
    if X.dtype == np.int or X.dtype == np.int64:
        X = X.fillna(value=-1)
    elif X.dtype == np.float:
        X = X.fillna(value=-1)
    elif X.dtype == np.bool:
        X = X.fillna(value=False)
    elif X.dtype == np.object:
        X = X.fillna(value='missing')
    else:
        print(X.dtype)
    return X
for feat in X.columns:
    X[feat] = fillna_unknown_dtype_col(X[feat])
for feat in C.columns:
    C[feat] = fillna_unknown_dtype_col(C[feat])

/usr/local/Caskroom/miniconda/base/envs/correlator/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (21,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,449,450,451,452,454,455,459,462,464,465,466,467,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,496,499,500,501,502,503,504,505,506,507,508,509,519,523,524,527,528,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,570,571,575,581,594,596,598,599,600,601,602,603,604,608,612,613,614,615,616,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,663,664,665,667,668,669,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,719,722,723,724,725,726,727,728,729,730,731,732,733,

## Synthetic HTE (Heterogeneous Treatment Effect)

In [23]:
p_covariate = .5  # probability of assigning a patient the new covariate
p_treatment = .5  # probability of assigning a patient the new treatment
max_pflip_strength = .8  # probability of "curing" a patient with the new covariate and new treatment that originally died
simulation_type = 'gaussian'  # Map from covariate * treatment to HTE strength

Y = df['mortality'].copy().astype(int)
y_prior = 1 - np.mean(Y)
print(f"Mean survival {y_prior}")
n = len(Y)
pflip_transform = None
covariate_vals = None
if simulation_type == 'bernoulli':
    pflip_transform = lambda covariates: covariates.copy() * max_pflip_strength
    covariate_vals = np.array([0, 1])
elif simulation_type == 'uniform':
    pflip_transform = lambda covariates: covariates.copy() * max_pflip_strength
    covariate_vals = np.linspace(.01, 1, 1000)
elif simulation_type == 'gaussian':
    pflip_transform = lambda x: np.exp(-1/(2*.05)*(x-.5)**2) * max_pflip_strength
    covariate_vals = np.linspace(0.01, 1, 1000)
pflip_vals = pflip_transform(covariate_vals)

idx = np.random.choice(np.arange(len(covariate_vals)), size=n, replace=True) # , p=[1 - p_covariate, p_covariate])
covariate = covariate_vals[idx]
pflips = pflip_vals[idx]
treatment = np.random.binomial(n=1, p=p_treatment, size=n)
for i in range(n):
    if covariate[i] > 0 and treatment[i] == 1 and Y[i] == 1:
        flip = np.random.binomial(n=1, p=pflips[i])
        Y[i] -= flip
covariate_name = 'New Covariate'
treatment_name = 'New Treatment'
C[covariate_name] = covariate.astype(float)
X[treatment_name] = treatment.astype(float)

# Analysis functions
def pflip_to_param(p_flip):
    odds_prior = y_prior / (1 - y_prior)
    y_posterior = y_prior + p_flip * (1 - y_prior)
    odds_posterior = y_posterior / (1 - y_posterior)
    logodds = np.log(odds_posterior / odds_prior)
    return logodds

Mean survival 0.9261431931016462


## Mise en Place

In [24]:
# Process data
one_hot_encoders = []
for feat in C.columns:
    try:
        C[feat] = C[feat].astype(float)
    except:
        #C.drop(feat, axis=1, inplace=True)
        #continue
        enc = OrdinalEncoder()#handle_unknown='ignore')
        C[feat] = enc.fit_transform(C[feat].values.reshape(-1, 1))
        one_hot_encoders.append(enc)
    #print(feat, np.percentile(C[feat].values, 19), np.percentile(C[feat].values, 90))
    C[feat].loc[C[feat] > np.percentile(C[feat].values, 99)] = np.percentile(C[feat].values, 99)
    C[feat].fillna(-1, inplace=True)

X = X.astype(float)
for feat in X.columns:
    if np.sum(X[feat]) < 100:
        X.drop(feat, axis=1, inplace=True)

contextual = C.copy()
explainable = X.copy()
X_full = pd.concat([X, C], axis=1)
print(X.shape, C.shape, X_full.shape)
C_train, C_test, X_train, X_test, X_all_train, X_all_test, mortality_train, mortality_test = train_test_split(
    contextual, explainable, X_full, Y.astype(float), test_size=0.25)
C_means = np.mean(C_train, axis=0)
C_stds  = np.std(C_train, axis=0)
C_train = (C_train - C_means) / C_stds
C_test  = (C_test  - C_means) / C_stds
C_train = C_train.loc[:, C_stds > 0.1]  # removes Calcium Chloride covariate
C_test  = C_test.loc[:, C_stds > 0.1]

covariate_i = C_train.columns.tolist().index(covariate_name)
treatment_i = X_train.columns.tolist().index(treatment_name)

/usr/local/Caskroom/miniconda/base/envs/correlator/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/Caskroom/miniconda/base/envs/correlator/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/Caskroom/miniconda/base/envs/correlator/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats i

(76540, 90) (76540, 25) (76540, 115)


## Explainable Boosting Machine (baseline)

In [36]:
# EBM
# look for all treatment-covariate interactions
interactions = list(product(range(treatment_i + 1), range(treatment_i + 1, (treatment_i + 1) + (covariate_i + 1)  + 1)))
ebm_hte = ebc(outer_bags=1, interactions=interactions, max_bins=32) 
ebm_hte.fit(X_all_train, mortality_train)
# Get estimated weights
feat_i = ebm_hte.get_params()['feature_names'].index(f"{treatment_name} x {covariate_name}")
explanation = ebm_hte.explain_global().data(feat_i)
weights = np.array(explanation['scores'])
hte_est = weights[1] - weights[0]
hte_est -= hte_est[0]  # center
cov_vals = np.array(explanation['right_names']).astype(float)
if len(cov_vals) > 2:
    cov_vals = (cov_vals[1:] + cov_vals[:-1]) / 2  # middle of bin
pflip_true = pflip_transform(cov_vals) 
hte_true = pflip_to_param(pflip_true)
# Evaluate
ebm_train_auroc = roc(mortality_train,  ebm_hte.predict_proba(X_all_train)[:, 1])
ebm_train_f1 = f1_score(mortality_train, ebm_hte.predict(X_all_train))
ebm_test_auroc = roc(mortality_test,  ebm_hte.predict_proba(X_all_test)[:, 1])
ebm_test_f1 = f1_score(mortality_test, ebm_hte.predict(X_all_test))
ebm_l2_hte = ((hte_true - hte_est)**2).mean()
print(f"EBM Train AUROC:\t{ebm_train_auroc}")
print(f"EBM Train F1:\t\t{ebm_train_f1}")
print(f"EBM Test AUROC:\t\t{ebm_test_auroc}")
print(f"EBM Test F1:\t\t{ebm_test_f1}")
print(f"EBM L2:\t\t\t{ebm_l2_hte}")

EBM Train AUROC:	0.9088257651044838
EBM Train F1:		0.2706875753920386
EBM Test AUROC:		0.8910881230039902
EBM Test F1:		0.26215022091310747
EBM L2:			0.9121528696554778


## Logistic Regression (Baseline)

In [38]:
# Logsitic
def get_paired_df(X_df, Y_df):
    XY_header = []
    XY_values = []
    for X_col in X_df.columns:
        for Y_col in Y_df.columns:
            XY_col = f"{X_col} x {Y_col}"
            XY_vals = X_df[X_col] * Y_df[Y_col]
            XY_header.append(XY_col)
            XY_values.append(XY_vals)
    XY_values = np.array(XY_values).T
    # pd.concat fails to join, here's a workaround
    full_header = X_df.columns.tolist() + Y_df.columns.tolist() + XY_header
    full_values = np.concatenate((X_df.values, Y_df.values, XY_values), axis=1)
    return pd.DataFrame(data=full_values, columns=full_header)

X_paired_train = get_paired_df(C_train, X_train)
X_paired_test = get_paired_df(C_test, X_test)
interaction_i = X_paired_train.columns.tolist().index('New Covariate x New Treatment')

logistic = LogisticRegression(penalty='none')
logistic.fit(X_paired_train, mortality_train)
logistic_coef = logistic.coef_[0, interaction_i]
hte_est = logistic_coef * covariate_vals
hte_true = pflip_to_param(pflip_vals)

logistic_train_auroc = roc(mortality_train, logistic.predict_proba(X_paired_train)[:, 1])
logistic_train_f1 = f1_score(mortality_train, logistic.predict(X_paired_train))
logistic_test_auroc = roc(mortality_test, logistic.predict_proba(X_paired_test)[:, 1])
logistic_test_f1 = f1_score(mortality_test, logistic.predict(X_paired_test))
logistic_l2_hte = ((hte_true - hte_est)**2).mean()
print(f"Log Train AUROC:\t{logistic_train_auroc}")
print(f"Log Train F1:\t\t{logistic_train_f1}")
print(f"Log Test AUROC:\t\t{logistic_test_auroc}")
print(f"Log Test F1:\t\t{logistic_test_f1}")
print(f"Log L2:\t\t\t{logistic_l2_hte}")

/usr/local/Caskroom/miniconda/base/envs/correlator/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Log Train AUROC:	0.9096169859951208
Log Train F1:		0.3294930875576037
Log Test AUROC:		0.8514194649092176
Log Test F1:		0.23712067748764995
Log L2:			0.7818690184669947


In [138]:
import torch
from tqdm import trange
from contextualized.regression import NGAM, MultivariateDataset

C_train_t = torch.tensor(C_train.values, dtype=torch.float)
X_train_t_pre = torch.tensor(X_train.values, dtype=torch.float)
X_train_t = torch.cat((X_train_t_pre, torch.ones(X_train_t_pre.shape[0], 1)), dim=1)
Y_train_t = torch.tensor(mortality_train.values, dtype=torch.float).unsqueeze(-1)
C_test_t = torch.tensor(C_test.values, dtype=torch.float)
X_test_t_pre = torch.tensor(X_test.values, dtype=torch.float)
X_test_t = torch.cat((X_test_t_pre, torch.ones(X_test_t_pre.shape[0], 1)), dim=1)
Y_test_t = torch.tensor(mortality_test.values, dtype=torch.float).unsqueeze(-1)

mse = lambda y_hat, y: (y_hat - y).pow(2).mean()
sigmoid = lambda x: 1 / (1 + torch.exp(-x))
ngam = NGAM(C_train_t.shape[-1], X_train_t.shape[-1], width=100, layers=3)
opt = torch.optim.Adam(ngam.parameters(), lr=1e-4)

epochs = 1
batch_size = 2
train_dataset = MultivariateDataset(C_train_t, X_train_t, Y_train_t, batch_size=batch_size)
progress_bar = tqdm(range(epochs))
for _ in progress_bar:
    for batch_i, (c, _, x, y) in enumerate(train_dataset):
        w = ngam(c)
        y_hat = sigmoid((w * x).sum(dim=1).squeeze())
        loss = mse(y_hat, y.squeeze())
        opt.zero_grad()
        loss.backward()
        opt.step()
        train_desc = f'[Train MSE: {loss.item():8.4f}] [Sample: {batch_size * batch_i}/{len(train_dataset)}]'
        progress_bar.set_description(train_desc)

/Users/calebellington/Workbench/ContextualizedCorrelator/contextualized/regression.py:128: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.C = torch.tensor(C, dtype=dtype, device=device)
/Users/calebellington/Workbench/ContextualizedCorrelator/contextualized/regression.py:129: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=dtype, device=device)
/Users/calebellington/Workbench/ContextualizedCorrelator/contextualized/regression.py:130: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.Y = torch.tensor(Y, dt

In [139]:
train_preds = sigmoid((X_train_t * ngam(C_train_t)).sum(dim=1)).squeeze().detach().numpy()
test_preds = sigmoid((X_test_t * ngam(C_test_t)).sum(dim=1)).squeeze().detach().numpy()
feature_model = ngam.nams[covariate_i]
covariate_inputs = (covariate_vals - C_means[covariate_name]) / C_stds[covariate_name]
feature_in = torch.tensor(covariate_inputs, dtype=torch.float).unsqueeze(-1)
feature_out = feature_model(feature_in).detach().numpy()
hte_est = feature_out[:,treatment_i]
hte_true = pflip_to_param(pflip_vals)

cgam_train_auroc = roc(mortality_train, train_preds)
cgam_train_f1 = f1_score(mortality_train, np.round(train_preds))
cgam_test_auroc = roc(mortality_test, test_preds)
cgam_test_f1 = f1_score(mortality_test, np.round(test_preds))
cgam_l2_hte = ((hte_true - hte_est)**2).mean()
print(f"CGAM Train AUROC:\t{cgam_train_auroc}")
print(f"CGAM Train F1:\t\t{cgam_train_f1}")
print(f"CGAM Test AUROC:\t{cgam_test_auroc}")
print(f"CGAM Test F1:\t\t{cgam_test_f1}")
print(f"CGAM L2:\t\t{cgam_l2_hte}")

CGAM Train AUROC:	0.8573239175825433
CGAM Train F1:		0.17815646785437644
CGAM Test AUROC:	0.8527916283390218
CGAM Test F1:		0.17795275590551182
CGAM L2:		0.9016688276012631
